In [51]:
import numpy
import pandas as pd 
import nltk
import string
import re
import os
import joblib
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [52]:
art_df = pd.read_csv('C:/Users/Powerhouse/Documents/GitHub/Project-High/article-database.csv')
print(art_df.shape)
art_df.head()

(18769, 6)


,Unnamed: 0,Name,Url,Text,Tags,Claps
0,0,None,https://medium.com/@elektra.sunn/cheshire-moon...,NaN,[],[]
1,0,None,https://medium.com/@nicolascole77/if-youre-doi...,NaN,[],[]
2,0,The Ultimate Travel Workout and Diet Plans by ...,https://medium.com/@garrettgee/the-bucket-list...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...",[]
3,0,Building a Responsive Image Gallery,https://medium.com/backticks-tildes/building-a...,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']",[]
4,0,When they’re already “forever” for you,https://medium.com/@krisgage/when-theyre-alrea...,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...",[]


In [53]:
model_db = art_df.drop([art_df.columns[0], art_df.columns[2], art_df.columns[5]], axis=1)
model_db.head()

,Name,Text,Tags
0,None,NaN,[]
1,None,NaN,[]
2,The Ultimate Travel Workout and Diet Plans by ...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
3,Building a Responsive Image Gallery,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
4,When they’re already “forever” for you,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."


In [54]:
model_db.isnull().sum()

Name       4
Text    1114
Tags       0
dtype: int64

In [55]:
model_db_clean = model_db.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
model_db_clean.head()

,Name,Text,Tags
0,The Ultimate Travel Workout and Diet Plans by ...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
1,Building a Responsive Image Gallery,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
2,When they’re already “forever” for you,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."
3,Forking The iPhone,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']"
4,Method swizzling in iOS swift,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']"


In [56]:
sum = 0
for i in range(0, model_db_clean.shape[0]):
    sum = sum + len(model_db_clean.Text[i])
sum/model_db_clean.shape[0]

6882.800011330803

In [57]:
# from ast import literal_eval
def func(raw_tags):
    raw_split = raw_tags[1:-1].split(',')
    num_tags = len(raw_split)
    tags_clean = []
    tags_clean.append(raw_split[0][1:-1])
    for i in range(1, num_tags):
       tags_clean.append(raw_split[i][2:-1])
    return tags_clean

model_db_clean['Tags_clean'] = model_db_clean['Tags'].apply(lambda x: func(x))

# model_db_clean.loc[:,'Tags'] = model_db_clean.loc[:,'Tags'].apply(lambda x: literal_eval(x))
print(type(model_db_clean['Tags']))
model_db_clean.head()

<class 'pandas.core.series.Series'>


,Name,Text,Tags,Tags_clean
0,The Ultimate Travel Workout and Diet Plans by ...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...","[Fitness, Travel, Diet, Exercise, Vacation]"
1,Building a Responsive Image Gallery,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']","[Design, Data, Growth, CSS, Web Design]"
2,When they’re already “forever” for you,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...","[Love, Relationships, Dating, Men, Women]"
3,Forking The iPhone,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']","[Publishing, Apple, Apple]"
4,Method swizzling in iOS swift,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']","[Swift, iOS, Apple]"


In [58]:
multi_label_transform = MultiLabelBinarizer()
multi_label_transform.fit(model_db_clean['Tags_clean'])

y = multi_label_transform.transform(model_db_clean['Tags_clean'])
print(y.shape, model_db_clean.shape)
print(y)

(17651, 913) (17651, 4)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [60]:
cols = []
for i in list(multi_label_transform.classes_):     
    cols.append(i)
cols.append('Text')

prepd_db = pd.DataFrame()

prepd_db.loc[:, 'Text'] = model_db_clean.loc[:, 'Text']
for i in range(0, y.shape[1]):
        prepd_db.loc[:, i+1] = y[:, i]

ps = PorterStemmer()
def clean_text(text):
    text = re.sub("\'", "", text) 
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower()
    _t = ""
    for t in text.split():
        _t += ps.stem(t) + " "
    text = _t

    return text

prepd_db['Text_clean'] = prepd_db['Text'].apply(lambda x: clean_text(x))
prepd_db.drop(['Text'], axis=1, inplace=True)

prepd_db.columns = cols
prepd_db.drop([''], axis=1, inplace=True)

prepd_db.head()

,2016,2016 Election,2017,3D Printing,AI,API,AWS,Abortion,Abuse,Academia,...,YouTube,Youth,Zen,eBooks,iOS,iOS App Development,iPad,iPhone,日本語,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introduc the bucket list famili hi we are the ...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in thi post you would learn to build a basic i...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,becaus love that s light can last that long al...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,by jean loui gass e a delay monday note due to...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,today i wa integr fcm in my io app i saw the s...


In [61]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

prepd_db['Text'] = prepd_db['Text'].apply(lambda x: remove_stopwords(x))
prepd_db.head()

,2016,2016 Election,2017,3D Printing,AI,API,AWS,Abortion,Abuse,Academia,...,YouTube,Youth,Zen,eBooks,iOS,iOS App Development,iPad,iPhone,日本語,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introduc bucket list famili hi bucket list fam...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thi post would learn build basic imag galleri ...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,becaus love light last long past partner frank...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jean loui gass e delay monday note due logist ...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,today wa integr fcm io app saw word method swi...


In [62]:
for i in range(0, prepd_db.shape[1]):
    x = prepd_db.iloc[:, i].value_counts()
    if x[1] > 500:
        print(x)

0    17132
1      519
Name: AI, dtype: int64
0    16689
1      962
Name: Blockchain, dtype: int64
0    17125
1      526
Name: Business, dtype: int64
0    17041
1      610
Name: Cryptocurrency, dtype: int64
0    17088
1      563
Name: Data Science, dtype: int64
0    16620
1     1031
Name: Design, dtype: int64
0    16767
1      884
Name: Entrepreneurship, dtype: int64
0    17139
1      512
Name: Health, dtype: int64
0    17147
1      504
Name: Humor, dtype: int64
0    15963
1     1688
Name: Life, dtype: int64
0    16349
1     1302
Name: Life Lessons, dtype: int64
0    16857
1      794
Name: Love, dtype: int64
0    16888
1      763
Name: Machine Learning, dtype: int64
0    17140
1      511
Name: Marketing, dtype: int64
0    17076
1      575
Name: News, dtype: int64
0    16918
1      733
Name: Poetry, dtype: int64
0    16427
1     1224
Name: Politics, dtype: int64
0    16950
1      701
Name: Productivity, dtype: int64
0    16966
1      685
Name: Programming, dtype: int64
0    17109
1      

In [65]:
y = []
for i in range(0, prepd_db.shape[1]):
    x = prepd_db.iloc[:, i].value_counts()
    if x[1] > 500:
        y.append(x.name)      
  
model_final_data = prepd_db[y]
model_final_data['Text'] = prepd_db['Text']

model_final_data.head()

,AI,Blockchain,Business,Cryptocurrency,Data Science,Design,Entrepreneurship,Health,Humor,Life,...,Self Improvement,Software Development,Sports,Startup,Tech,Technology,UX,Web Development,Writing,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introduc bucket list famili hi bucket list fam...
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thi post would learn build basic imag galleri ...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,becaus love light last long past partner frank...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jean loui gass e delay monday note due logist ...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,today wa integr fcm io app saw word method swi...


In [66]:
model_final_data.to_csv('model-data.csv')

In [67]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=1000)
train_tfidf = tfidf_vectorizer.fit_transform(model_final_data['Text'])

# for x in range(0, len(model_final_data.columns)-1):
#     xtrain, xval, ytrain, yval = train_test_split(model_final_data['Text'], model_final_data.iloc[:, x], test_size=0.2, random_state=9)

#     xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
#     xval_tfidf = tfidf_vectorizer.transform(xval)

#     lr = LogisticRegression(tol=0.3)
#     lr.fit(xtrain_tfidf, ytrain)

#     joblib.dump(lr, 'model_pickle_files/' + str(model_final_data.columns[x]) + '.pkl')

for x in range(0, len(model_final_data.columns)-1):

    lr = LogisticRegression(tol=0.75)
    lr.fit(train_tfidf, model_final_data.iloc[:, x])

    joblib.dump(lr, 'model_pickle_files/' + str(model_final_data.columns[x]) + '.pkl')